@Set up

In [1]:
!pip install keras
!pip install pydub
!pip install pydot

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 20

In [1]:
import tensorflow as tf
import numpy as np

from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random 

In [4]:
path1 = os.path.join('./spectrograms/')
os.makedirs(path1)

os.mkdir('./spectrograms/train')
os.mkdir('./spectrograms/test')

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#training



In [23]:
genres = 'rythmandblues country disco hiphop jazz rock'
genres = genres.split()

In [24]:
print(genres)

['rythmandblues', 'country', 'disco', 'hiphop', 'jazz', 'rock']


In [33]:
#os.makedirs('./spectrogram_gen')
os.makedirs('./spectrogram_gen/train')
os.makedirs('./spectrogram_gen/test')

for g in genres:
  #path_audio = os.path.join('/content/audio3sec',f'{g}')
  #os.makedirs(path_audio)
  path_train = os.path.join('../dataset/spectrogram_gen/train/',f'{g}')
  path_test = os.path.join('../dataset/spectrogram_gen/test/',f'{g}')
  os. makedirs(path_train)
  os. makedirs(path_test)

FileExistsError: [Errno 17] File exists: '../dataset/spectrogram_gen/train/rythmandblues'

In [6]:
directory = os.path.join('./spectrograms3sec/')
for g in genres:
  print(g)
  for filename in os.listdir(directory + f"{g}"):
    print(filename)
    #print(1)
    destination = os.path.join('./spectrogram_gen/train/'+ f"{g}")
    shutil.copy(directory + f"{g}"+"/"+filename, destination)

rythmandblues
rythmandblues585.png
rythmandblues591.png
rythmandblues208.png
rythmandblues220.png
rythmandblues546.png
rythmandblues552.png
rythmandblues234.png
rythmandblues963.png
rythmandblues977.png
rythmandblues787.png
rythmandblues793.png
rythmandblues778.png
rythmandblues744.png
rythmandblues750.png
rythmandblues988.png
rythmandblues817.png
rythmandblues803.png
rythmandblues195.png
rythmandblues181.png
rythmandblues618.png
rythmandblues630.png
rythmandblues156.png
rythmandblues142.png
rythmandblues624.png
rythmandblues397.png
rythmandblues56.png
rythmandblues42.png
rythmandblues383.png
rythmandblues368.png
rythmandblues354.png
rythmandblues432.png
rythmandblues95.png
rythmandblues426.png
rythmandblues81.png
rythmandblues340.png
rythmandblues80.png
rythmandblues427.png
rythmandblues341.png
rythmandblues355.png
rythmandblues94.png
rythmandblues433.png
rythmandblues369.png
rythmandblues43.png
rythmandblues382.png
rythmandblues396.png
rythmandblues57.png
rythmandblues143.png
rythman

In [7]:
directory = "./spectrogram_gen/train/"
for g in genres:
  filenames = os.listdir(os.path.join(directory,f"{g}"))
  random.shuffle(filenames)
  test_files = filenames[0:100]

  for f in test_files:

    shutil.move(directory + f"{g}"+ "/" + f,"./spectrogram_gen/test/" + f"{g}")


In [35]:
train_dir = "../dataset/spectrogram_gen/train/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(288,432),color_mode="rgba",
                                                    class_mode='categorical',batch_size=128)

validation_dir = "../dataset/spectrogram_gen/test/"
vali_datagen = ImageDataGenerator(rescale=1./255)
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(288,432),color_mode='rgba',
                                                  class_mode='categorical',batch_size=128)

Found 2470 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


#Model

In [4]:
def GenreModel(input_shape = (288,432,4),classes=6):
  np.random.seed(6)
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1),kernel_initializer = glorot_uniform(seed=9))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1),kernel_initializer=glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1),kernel_initializer = glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1),kernel_initializer=glorot_uniform(seed=9))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)

  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=9))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model


In [5]:
import keras.backend as K
import tensorflow as tf

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [6]:
model = GenreModel(input_shape = (288,432,4), classes=6)
opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

2021-11-16 22:35:06.356972: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [44]:
model.load_weights("../musicarts.h5")

In [8]:
model.summary()

Model: "GenreModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 288, 432, 4)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 286, 430, 8)       296       
_________________________________________________________________
batch_normalization (BatchNo (None, 286, 430, 8)       32        
_________________________________________________________________
activation (Activation)      (None, 286, 430, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 141, 213, 16)      1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 213, 16)      6

In [ ]:
model.fit_generator(train_generator,epochs=70,validation_data=vali_generator)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-11-05 13:56:35.493225: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/70
43/43 [==============================] - 401s 9s/step - loss: 3.0767 - accuracy: 0.2543 - get_f1: 0.1978 - val_loss: 1.8305 - val_accuracy: 0.1667 - val_get_f1: 0.0000e+00
Epoch 2/70
43/43 [==============================] - 211s 5s/step - loss: 1.3254 - accuracy: 0.4728 - get_f1: 0.3524 - val_loss: 2.0497 - val_accuracy: 0.2717 - val_get_f1: 0.0000e+00
Epoch 3/70
43/43 [==============================] - 204s 5s/step - loss: 1.1519 - accuracy: 0.5509 - get_f1: 0.4991 - val_loss: 2.5672 - val_accuracy: 0.1683 - val_get_f1: 0.1804
Epoch 4/70
43/43 [==============================] - 203s 5s/step - loss: 0.9509 - accuracy: 0.6452 - get_f1: 0.5995 - val_loss: 2.8642 - val_accuracy: 0.1667 - val_get_f1: 0.1665
Epoch 5/70
43/43 [==============================] - 229s 5s/step - loss: 0.8239 - accuracy: 0.6983 - get_f1: 0.6718 - val_loss: 3.1390 - val_accuracy: 0.1783 - val_get_f1: 0.1748
Epoch 6/70
43/43 [==============================] - 206s 5s/step - loss: 0.7583 - accuracy: 0.718

In [ ]:
loss, acc, fi = model.evaluate(vali_generator,batch_size=128)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
Y_test = []
y_pred = [] # Convert one-hot to index

In [45]:
predIdxs = model.predict(vali_generator, batch_size=128)

In [46]:
predIdxs = np.argmax(predIdxs, axis=1)

In [41]:
class_labels = ['rhythmandblues', 'country', 'disco', 'hiphop', 'jazz', 
                'rock']

In [47]:

print(classification_report(vali_generator.labels, predIdxs,
                            target_names=class_labels))

                precision    recall  f1-score   support

rhythmandblues       0.16      0.24      0.19       100
       country       0.13      0.16      0.14       100
         disco       0.18      0.10      0.13       100
        hiphop       0.19      0.16      0.17       100
          jazz       0.19      0.15      0.17       100
          rock       0.14      0.15      0.14       100

      accuracy                           0.16       600
     macro avg       0.16      0.16      0.16       600
  weighted avg       0.16      0.16      0.16       600



In [ ]:
print(len(model.layers))

19


In [ ]:
loss, acc, fi = model.evaluate(vali_generator,batch_size=128)

In [ ]:
print("loss is", loss)
print("acc is", acc)

loss is 0.6670132279396057
acc is 0.8199999928474426


In [ ]:
model.save('genre.h5')